In [1]:
import pandas as pd

# read in

In [2]:
df = pd.read_csv('../../2_Feature_Engineering/export/combined_polly_bretschneider_iwg.csv')
df.head()

,raw_text,severity,binary_label
0,ist ekelhaft bei ihren Klos kannst du nur mit...,0.0,False
1,Bevor Sie mit Jud...,1.0,True
2,Und de Amis freut es wenn die Heuschrecken Eur...,0.0,False
3,": Die Krim ist ein heißer Tip, auch zum In...",1.0,True
4,: Es muss in #Deutschland eine politische Kra...,1.0,True


# clean

In [3]:
import sys
sys.path.insert(0, r'..\..\0_common')
from model_helpers import clean_all as ci

In [4]:
df['cleaned_text'] = df['raw_text'].apply(ci)
df.head()

,raw_text,severity,binary_label,cleaned_text
0,ist ekelhaft bei ihren Klos kannst du nur mit...,0.0,False,ekelhaft klos kannst gummistiefel gehen stinkt...
1,Bevor Sie mit Jud...,1.0,True,bevor judenstern markiert worden juden ganze ...
2,Und de Amis freut es wenn die Heuschrecken Eur...,0.0,False,de amis freut heuschrecken europa klein wirtsc...
3,": Die Krim ist ein heißer Tip, auch zum In...",1.0,True,krim heißer tip investieren muß bürger landes...
4,: Es muss in #Deutschland eine politische Kra...,1.0,True,deutschland politische kraft geben sozial sch...


In [5]:
df.shape

(9169, 4)

# vectorize

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
vec = TfidfVectorizer(ngram_range=(1,3), analyzer='word', max_features=100000)

In [8]:
X_features = vec.fit_transform(df['cleaned_text'])

In [9]:
X_features.shape

(9169, 100000)

# initialize model

In [10]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_jobs=-1, n_estimators=100)

# Cross validation

In [11]:
from sklearn.model_selection import cross_validate
scores = cross_validate(clf, X_features, df['binary_label'], cv=5,
                        scoring=('f1', 'roc_auc', 'precision', 'recall'),
                        return_train_score=True)

In [12]:
sorted(scores.keys())

['fit_time',
 'score_time',
 'test_f1',
 'test_precision',
 'test_recall',
 'test_roc_auc',
 'train_f1',
 'train_precision',
 'train_recall',
 'train_roc_auc']

In [13]:
scores_df = pd.DataFrame(scores)
scores_df

,fit_time,score_time,test_f1,train_f1,test_roc_auc,train_roc_auc,test_precision,train_precision,test_recall,train_recall
0,25.800985,0.931886,0.874310,0.999710,0.957783,0.999860,0.929596,1.0,0.825231,0.999421
1,22.809026,0.854023,0.886392,0.999566,0.966340,0.999896,0.962060,1.0,0.821759,0.999132
2,22.381125,0.848690,0.849591,0.999710,0.956409,0.999853,0.931034,1.0,0.781250,0.999421
3,22.519758,0.857068,0.881188,0.999566,0.967179,0.999856,0.946809,1.0,0.824074,0.999132
4,23.253777,0.865565,0.873846,0.999711,0.967475,0.999868,0.931759,1.0,0.822711,0.999421


In [14]:
# metrics
scores_df[['test_f1', 'test_roc_auc', 'test_precision', 'test_recall']].mean()

test_f1           0.873065
test_roc_auc      0.963037
test_precision    0.940251
test_recall       0.815005
dtype: float64